## 导入函数库及设定参数

In [ ]:
# 导入函数库
from jqdata import *

MA_WIN_1 = 10
MA_WIN_2 = 30

ATR_WIN_SIZE = 20  
RISK_RATIO = 0.001 

# 当日开盘价相对于前一次买入价的盈利比阈值
INC_POS_PF_RATE = 0.05
# 这里和上面的5%都是经验值
MAX_DROP_RATE = 0.03 

## 初始化函数

设定基准、税点、手续费及一些全局变量

In [ ]:
# 初始化函数，设定基准等等
def initialize(context):
    
    set_benchmark('000300.XSHG')
    set_option('use_real_price', True)
    
    # log.set_level('order', 'error')
    
    set_order_cost(OrderCost(close_tax=0.001, open_commission=0.0003, close_commission=0.0003, min_commission=5), type='stock')
    
    # 开盘前运行
    run_daily(before_market_open, time='before_open',
                reference_security='000300.XSHG')
    run_daily(market_open, time='every_bar',
                reference_security='000300.XSHG')
    run_daily(after_market_close, time='after_close',
                reference_security='000300.XSHG')
    # 设定股票池
    g.stock_pool = get_index_stocks("000016.XSHG", date=context.current_dt)
    g.init_cash = context.portfolio.starting_cas
    # 加仓：用于记录入场价格
    g.last_entry_prices = {code:None for code in g.stock_pool}
    # 浮动止损：用于记录入场时间
    g.entry_dates = {code:None for code in g.stock_pool}

## 开盘前运行的函数

确定买卖信号并在开盘前检测

In [ ]:
## 开盘前运行函数     
def before_market_open(context):
    look_ahead_n = max(MA_WIN_1, MA_WIN_2) + 1
    g.up_cross_signaled = set()
    g.down_cross_signaled = set()
    for code in g.stock_pool:
        df = attribute_history(code, look_ahead_n, "1d", ["close"],
            skip_paused=True) # 该函数返回结果不包括当天的数据
        
        if len(df) != look_ahead_n:
            continue
        close = df['close']
        ma_short = pd.rolling_mean(close, MA_WIN_1)
        ma_long = pd.rolling_mean(close, MA_WIN_2)
        
        uc_flags = (ma_short.shift(1) <= ma_long.shift(1)) & \
                    (ma_short > ma_long)
        dc_flags = (ma_short.shift(1) >= ma_long.shift(1)) & \
                    (ma_short < ma_long)
        
        if uc_flags.iloc[-1]:
            g.up_cross_signaled.add(code) 
        if dc_flags.iloc[-1]:
            g.down_cross_signaled.add(code)

## 开盘时运行的函数

- 检测死叉信号中的股票，卖出并重置标的最后入场价格和时间

- 检测金叉信号股票
    - 计算ATR，确定标的头寸规模，买入并记录最后入场价格和时间
    
- 对持仓股进行检测(加仓和止损只能二选一)
    - 是否符合加仓条件
        - 若符合，则计算ATR，确定加仓的头寸规模，记录最后入场价格和时间
    - 是否符合浮动止损条件
        - 若符合，则卖出标的，并重置标的最后入场价格和时间
 


In [ ]:
## 开盘时运行函数
def market_open(context):
    cur_dt = context.current_dt.date()
    p = context.portfolio
    current_data = get_current_data()
    # each_cash = g.init_cash / len(g.stock_pool) # 等金额分配资金
    
    # 卖出均线死叉信号的持仓股
    for code, pos in p.positions.items():
        if code in g.down_cross_signaled:
            order_target(code, 0)
            # 均匀加仓：记录最后一个入场价格
            g.last_entry_prices[code] = None
            # 浮动止损：记录入场时间
            g.entry_dates[code] = None
    
    # 买入均线金叉信号的持仓股
    for code in g.up_cross_signaled:
        if code not in p.positions:
            if current_data[code].paused:
                continue
            
            # 计算ATR - code start
            df = attribute_history(code, ATR_WIN_SIZE + 1, "1d", 
                ['high', 'low', 'close'], skip_paused=True)
            
            if len(df) != ATR_WIN_SIZE + 1:
                continue
            
            df['pdc'] = df['close'].shift(1)
            tr = df.apply(lambda x : max( x['high'] - x['low'], abs(x["high"] - x["pdc"]), abs(x['low'] - x['pdc'])), axis=1)
            atr = tr[-ATR_WIN_SIZE:].mean()
            num_to_buy = g.init_cash * RISK_RATIO / atr // 100 * 100
            # 计算ATR -- code end
            
            open_price = current_data[code].day_open
            # num_to_buy = each_cash / open_price // 100 * 100
            order(code, num_to_buy)
            # 均匀加仓：记录最后一个入场价格
            g.last_entry_prices[code] = open_price
            # 浮动止损: 记录进入市场的时间
            g.entry_dates[code] = cur_dt
    
    # if 检查有无符合加仓条件的持仓股
    # elif 检查有无符合回撤止盈条件的持仓股
    for code, pos in p.positions.items():
        # 过滤掉停牌的股票
        if current_data[code].paused:
            continue
        if pos.today_amount == 0 and pos.closeable_amount > 0:
            
            # 用于加仓 -- start
            open_price = current_data[code].day_open
            last_entry = g.last_entry_prices[code]
            # 用于加仓 -- end
            
            # 用于回撤止盈
            last_entry_date = g.entry_dates[code]
            prev_date = context.current_dt - timedelta(days=1)
            # 计算截止到前一天的HHV, 避免未来函数, 考虑到前复权, 每个交易日都重新计算
            df = get_price(code, start_date=last_entry_date, end_date=prev_date,
                frequency="1d", fields=['high'], skip_paused=True)
            hhv = df['high'].max()
            # 以当日开盘价计算，或者，也可以用前一交易日的收盘价计算
            drop_rate = (hhv - current_data[code].day_open) / hhv
            
            
            # 用于加仓
            if (open_price - last_entry) / last_entry >= INC_POS_PF_RATE:
                
                # 计算ATR - code start
                df = attribute_history(code, ATR_WIN_SIZE + 1, "1d", 
                    ['high', 'low', 'close'], skip_paused=True)
                
                if len(df) != ATR_WIN_SIZE + 1:
                    continue
                
                df['pdc'] = df['close'].shift(1)
                tr = df.apply(lambda x : max( x['high'] - x['low'], abs(x["high"] - x["pdc"]), abs(x['low'] - x['pdc'])), axis=1)
                atr = tr[-ATR_WIN_SIZE:].mean()
                
                # 计算ATR -- code end
                
                # order_value(code, each_cash)
                num_to_buy = g.init_cash * RISK_RATIO / atr // 100 * 100
                order(code, num_to_buy)
                
                # 用于加仓：记录入场价格
                g.last_entry_prices[code] = open_price
                # 用于止损 -- 记录入场时间
                g.entry_dates[code] = cur_dt
                
            elif drop_rate > MAX_DROP_RATE:
                order_target(code, 0)
                # 均匀加仓：记录最后一个入场价格
                g.last_entry_prices[code] = None
                # 止损 -- 记录入场时间
                g.entry_dates[code] = None

## 收盘后运行的函数

计算持仓，并在回测中展示出来

In [ ]:
## 收盘后运行函数  
def after_market_close(context):
    p = context.portfolio
    pos_level = p.positions_value / p.total_value
    record(pos_level=pos_level)